# CRAWL STOCK API

## Nguồn dữ liệu: DStock, FireAnt, VietStock Finance

### 1. Chuẩn bị dữ liệu

Import các thư viện cần thiết cho việc thu thập dữ liệu.

In [3]:
import requests
import pandas as pd

Lấy thông tin các công ty đã từng được đưa lên sàn chứng khoán Việt Nam

In [1]:
def get_stock_list():
    VNDIRECT_API = 'https://finfo-api.vndirect.com.vn/v4/stocks'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
        'Content-Type': 'application/json',
    }
    
    params = {
        "q": "type:stock,ifc~floor:HOSE,HNX,UPCOM",
        "size": "9999"
    }
    
    response = requests.get(url = VNDIRECT_API, params = params, headers = headers)
    
    if response.status_code == 200:
        field_to_get = ["code", "type", "status", "companyName", "listedDate", "delistedDate"]
        df = pd.DataFrame(response.json()["data"])
        df = df[field_to_get]
        return pd.DataFrame(df)
        
    return []

In [4]:
stock_data = get_stock_list()
stock_data.head()

,code,type,status,companyName,listedDate,delistedDate
0,ENF,IFC,delisted,Quỹ Đầu tư Năng động Eastspring Investments Vi...,2001-01-01,2001-01-01
1,FUCVREIT,IFC,listed,Quỹ đầu tư bất động sản Techcom Việt Nam,2017-02-27,NaN
2,FUCTVGF2,IFC,delisted,Quỹ Đầu tư Tăng trưởng Thiên Việt 2,2018-09-26,2022-02-25
3,ASIAGF,IFC,delisted,Quỹ đầu tư tăng trưởng ACB,2012-06-26,2014-12-30
4,PRUBF1,IFC,delisted,Quỹ đầu tư cân bằng Prudential,2006-12-04,2013-09-03


Data clean up

In [5]:
import re

def cleanRoman(text):
    pattern = r'\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?'
    return re.sub(pattern, '', text)

def cleanText(text):
    pattern = r'[^A-Za-z]+'
    return re.sub(pattern, '', text)

def cleanBullet(text):
    pattern = '\w[.)]\s*'
    return re.sub(pattern, '', text)

def removeVietNameAccent(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

def removeSpace(text):
    pattern = r'\s*'
    return re.sub(pattern, '', text)

Lấy các chỉ số báo cáo tài chính

In [43]:
import datetime

def get_balance_sheet(symbol = "VNM", fromYear = 2021, toYear = 2022):
    companyBalanceSheet = []
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    }
    fireant_bearer_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoxOTM5NDc0NDY3LCJuYmYiOjE2Mzk0NzQ0NjcsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsib3BlbmlkIiwicHJvZmlsZSIsInJvbGVzIiwiZW1haWwiLCJhY2NvdW50cy1yZWFkIiwiYWNjb3VudHMtd3JpdGUiLCJvcmRlcnMtcmVhZCIsIm9yZGVycy13cml0ZSIsImNvbXBhbmllcy1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImZpbmFuY2UtcmVhZCIsInBvc3RzLXdyaXRlIiwicG9zdHMtcmVhZCIsInN5bWJvbHMtcmVhZCIsInVzZXItZGF0YS1yZWFkIiwidXNlci1kYXRhLXdyaXRlIiwidXNlcnMtcmVhZCIsInNlYXJjaCIsImFjYWRlbXktcmVhZCIsImFjYWRlbXktd3JpdGUiLCJibG9nLXJlYWQiLCJpbnZlc3RvcGVkaWEtcmVhZCJdLCJzdWIiOiJkM2UxY2I4MC0xMDc0LTRhMjItYWY4Ny0yNjlhOGM3Mzc2NmMiLCJhdXRoX3RpbWUiOjE2Mzk0NzQ0NjcsImlkcCI6Ikdvb2dsZSIsIm5hbWUiOiJtaW5odHJpLm1pbmh6enh6eEBnbWFpbC5jb20iLCJzZWN1cml0eV9zdGFtcCI6ImIzNDM3MmFkLTgxZjktNGUyYy04NTc4LTBmYWE3NmIxYmMzOSIsInByZWZlcnJlZF91c2VybmFtZSI6Im1pbmh0cmkubWluaHp6eHp4QGdtYWlsLmNvbSIsInVzZXJuYW1lIjoibWluaHRyaS5taW5oenp4enhAZ21haWwuY29tIiwiZnVsbF9uYW1lIjoiTWluaCBUcmkgTmd1eWVuIiwiZW1haWwiOiJtaW5odHJpLm1pbmh6enh6eEBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6InRydWUiLCJqdGkiOiIzY2FjMTQwZGIxMTRkNGMwOWI2MWJjNTA1NmQ0MDg0OCIsImFtciI6WyJleHRlcm5hbCJdfQ.X9deVcDttd06BxdZC7uOBXeObi3qOYqIsWK190UXRBSbVw-03W4KlsQ5PwKyoAc5beog9zYTtZzoE63cnbJ4o14aq4ljsM4bcFEfP2wLl3taVjuKbJOKaFMLiUFyQGiPc5_iE7b-7Z3cVWyEWtDl9xeqg57vVrBLXvcyzquWTFVKgaumR7PA3EwM5UHQWL8f2nx_zwAW06Y-x6soQItu8byN4Brm6VZK6YawUikZqsNehRxHmd_Q52rd4WJ5cTnLUHSlHNoKzEVOobfvOStE2bkoEceBuwgnjEIgqvFsdEX26lvi7ytkkUad9_Mm4LIs_-MxAnsoop3K0IFMzgq-IQ"
    headers.update({'Authorization': f"Bearer {fireant_bearer_token}"})
    
    FIREANT_API = f"https://restv2.fireant.vn/symbols/{symbol}/full-financial-reports?"
    
    field_to_get = [
        { 'tongcongtaisan': 'totalAssets'},
        { 'taisancodinhhuuhinh': 'tangibleAssets'},
        { 'taisancodinhvohinh': 'intangibleAssets'},
        { 'doanhthuthuan': 'netRevenue'},
        { 'loinhuantruocthue': 'profitBeforeTaxes'},
        { 'loinhuansauthuecuacodongcuacongtyme': 'profitAfterTaxes'},
        { 'tonghangtonkho': 'inventory'},
        { 'nophaitra': 'liabilities'},
        { 'tienvatuongduongtiencuoiky': 'cashAndCashEquivalents'},
        { 'vonchusohuu': 'equity'},
        { 'nonganhan': 'shorttermLiabilities'},
        { 'nodaihan': 'longtermLiabilities'},
        { 'giavonhangban': 'costPrice'},
        { 'khauhaotscd': 'fixedAssetsDepreciation'},
        { 'trongdochiphilaivay': 'lendingCost'},
        { 'vayvanothuetaichinhnganhan': 'shorttermBorrowingsFinancialLeases'},
        { 'vayvanothuetaichinhdaihan': 'longtermBorrowingsFinancialLeases'},
    ]
    
    for year in range(fromYear, toYear+1):
        for quarter in range(1, 4+1):
            if datetime.datetime(year, quarter*3, 1) > datetime.datetime.now():
                continue
            
            print(f"Getting data for {symbol} - {year} - {quarter}")
                  
            quarterBalanceSheet = {}
                                    
            for field in field_to_get:
                quarterBalanceSheet.update({list(field.values())[0] : 0})
             
            # 1  : "candoiketoan"
            # 2  : "ketquakinhdoanh"
            # 3  : "luuchuyentientett"
            # 4  : "luuchuyentientegt"
            for statementType in range(1, 5):                    
                params = {
                    "type": statementType, 
                    "year": year,
                    "quarter": quarter,
                    "limit": 1,
                }
                
                fireant_response = requests.get(FIREANT_API, headers=headers, params=params).json()
                                
                try:
                    if (fireant_response != None):
                        for field_respone in fireant_response:
                            if ('name' not in field_respone): continue
                            
                            field_name =  removeSpace(cleanText(removeVietNameAccent(cleanBullet(cleanRoman(field_respone['name']))))).lower()
                            field_value = field_respone['values'][0]['value'] or 0
                            
                            for field in field_to_get:
                                if field_name in field: quarterBalanceSheet.update({field[field_name]: str(field_value)})
                    
                    quarterBalanceSheet.update({f"year": year})
                    quarterBalanceSheet.update({f"quarter": quarter})
                    
                except:
                    print(f"{symbol} - {year} - {quarter} - {statementType} - {fireant_response}")
                    raise
                
            companyBalanceSheet.append(quarterBalanceSheet)
    return companyBalanceSheet

Lấy chỉ số báo cáo tài chính của công ty

In [44]:
print(get_balance_sheet())

Getting data for VNM - 2021 - 1
Getting data for VNM - 2021 - 2
Getting data for VNM - 2021 - 3
Getting data for VNM - 2021 - 4
Getting data for VNM - 2022 - 1
Getting data for VNM - 2022 - 2
Getting data for VNM - 2022 - 3
[{'totalAssets': '51051210880651.0', 'tangibleAssets': '12450177265747.0', 'intangibleAssets': '1131385809125.0', 'netRevenue': '13190270122852.0', 'profitBeforeTaxes': '3153910222805.0', 'profitAfterTaxes': '2575916578653.0', 'inventory': '6465943104329.0', 'liabilities': '17087270259224.0', 'cashAndCashEquivalents': '1197956795582.0', 'equity': '33963940621427.0', 'shorttermLiabilities': '16532816332955.0', 'longtermLiabilities': '554453926269.0', 'costPrice': '7435389686438.0', 'fixedAssetsDepreciation': '561074222639.0', 'lendingCost': '12905990201.0', 'shorttermBorrowingsFinancialLeases': '9247097278828.0', 'longtermBorrowingsFinancialLeases': '162103516784.0', 'year': 2021, 'quarter': 1}, {'totalAssets': '53046669029192.0', 'tangibleAssets': '12101578863690.0'

In [45]:
mask_active_stock_comapny = (stock_data['status'] == 'listed') & (stock_data['type'] == 'STOCK')
active_company = stock_data[mask_active_stock_comapny]
active_company

,code,type,status,companyName,listedDate,delistedDate
13,VCC,STOCK,listed,Công ty Cổ phần Vinaconex 25,2009-02-05,NaN
14,CTB,STOCK,listed,Công ty Cổ phần Chế tạo Bơm Hải Dương,2006-10-10,NaN
15,TTT,STOCK,listed,Công ty Cổ phần Du lịch - Thương mại Tây Ninh,2017-06-02,NaN
16,GDW,STOCK,listed,Công ty Cổ phần Cấp nước Gia Định,2018-12-18,NaN
17,SD9,STOCK,listed,Công ty Cổ phần Sông Đà 9,2006-12-20,NaN
...,...,...,...,...,...,...
1887,TTB,STOCK,listed,Công ty Cổ phần Tập đoàn Tiến Bộ,2018-08-17,NaN
1888,POW,STOCK,listed,Tổng Công ty Điện lực Dầu khí Việt Nam – Công ...,2019-01-14,NaN
1889,CTD,STOCK,listed,Công ty Cổ phần Xây dựng Coteccons,2010-01-20,NaN
1890,BFC,STOCK,listed,Công ty Cổ phần Phân bón Bình Điền,2015-10-07,NaN


In [47]:
# get balance sheet for all active company and save to csv file

for index, row in active_company.iterrows():
    symbol = row['code']
    company_balance_sheet = get_balance_sheet(symbol)
    
    pd.DataFrame(company_balance_sheet).to_csv(f"{symbol}.csv", index=False)

Getting data for VCC - 2021 - 1
Getting data for VCC - 2021 - 2
Getting data for VCC - 2021 - 3
Getting data for VCC - 2021 - 4
Getting data for VCC - 2022 - 1
Getting data for VCC - 2022 - 2
Getting data for VCC - 2022 - 3
